In [1]:
def conj_set_preimage(self, other):
    from itertools import product
    from copy import deepcopy
    f = copy(self)
    g = copy(other)
    n = f.domain().dimension_relative()
    if (n == 1) and (R in NumberFields() or R in FiniteFields())\
        and (f.sigma_invariants(1) != g.sigma_invariants(1)):
        return []
    
    def isLinearlyIndependent(S):
        M = matrix([list(t) for t in S])
        if len(S) < n+1:
            if M.rank() == len(S):
                return True
            return False
        else:
            if any(l == 0 for l in M.minors(n+1)):
                return False
            return True

    L = Set(f.periodic_points(1))
    K = Set(g.periodic_points(1))
    if len(L) != len(K):  # checks maps have the same number of fixed points
        return []
    mult_to_point_L = {}
    mult_to_point_K = {}
    point_to_mult_L = {}
    point_to_mult_K = {}
    level = 0
    for i in range(len(L)):
        mult_L = f.multiplier(L[i], 1).charpoly()
        mult_K = g.multiplier(K[i], 1).charpoly()
        tup_L = (mult_L, level)
        tup_K = (mult_K, level)
        if tup_L not in mult_to_point_L:
            mult_to_point_L[tup_L] = [L[i]]
        else:
            mult_to_point_L[tup_L] += [L[i]]
        if tup_K not in mult_to_point_K:
            mult_to_point_K[tup_K] = [K[i]]
        else:
            mult_to_point_K[tup_K] += [K[i]]
        point_to_mult_L[L[i]] = (mult_L, level)
        point_to_mult_K[K[i]] = (mult_K, level)
    #print('mult_to_point_L:', mult_to_point_L)
    #print('mult_to_point_K:', mult_to_point_K)
    #print('point_to_mult_L:', point_to_mult_L)
    #print('point_to_mult_K:', point_to_mult_K)
    repeated_mult_L = {}
    for mult_L in mult_to_point_L:
        repeated = len(mult_to_point_L[mult_L])
        if mult_L not in mult_to_point_K:
            return []
        elif len(mult_to_point_K[mult_L]) != repeated:
            return []
        if repeated not in repeated_mult_L:
            repeated_mult_L[repeated] = [mult_to_point_L[mult_L]]
        else:
            repeated_mult_L[repeated] += [mult_to_point_L[mult_L]]
    #print('repeated_mult:', repeated_mult_L)
    d = len(L)
    r = f.domain().base_ring()
    more = True
    source = []
    corresponding = []
    for r in sorted(repeated_mult_L.keys()):
        for point_lst in repeated_mult_L[r]:
            for point in point_lst:
                if isLinearlyIndependent(source + [point]):
                    source.append(point)
                    mult = point_to_mult_L[point]
                    # if another point with this multiplier and level pair is in S
                    # then the multiplier level pair will be the last element of corresponding
                    if len(corresponding) != 0:
                        if corresponding[-1][0] == mult:
                            corresponding[-1][1] += 1
                        else:
                            corresponding.append([mult, 1])
                    else:
                        corresponding.append([mult, 1])
                if len(source) == n+2:
                    more = False
                    break
            if len(source) == n+2:
                break
        if len(source) == n+2:
            break
    #print('corresponding:', corresponding)
    i_repeated_mult = deepcopy(repeated_mult_L)
    a_repeated_mult = {}
    found_no_more = True
    while more:
        level += 1
        for r in i_repeated_mult:
            for point_lst_L in i_repeated_mult[r]:
                old_tup_L = point_to_mult_L[point_lst_L[0]]
                #print('current tup:', old_tup_L)
                point_lst_K = mult_to_point_K[old_tup_L]
                mult_L = old_tup_L[0]
                Tl = []
                Tk = []
                for pnt in point_lst_L:
                    for preimage in f.rational_preimages(pnt):
                        if preimage != pnt:
                            Tl.append(preimage)
                for pnt in point_lst_K:
                    for preimage in g.rational_preimages(pnt):
                        if preimage != pnt:
                            Tk.append(preimage)
                #print('Tl:', Tl)
                #print('Tk:', Tk)
                if len(Tl) != len(Tk):
                    return []
                if len(Tl) != 0:
                    found_no_more = False
                    new_tup_L = (mult_L, level)
                    new_tup_K = (mult_L, level)
                    #print('new_tup_L:', new_tup_L)
                    mult_to_point_L[new_tup_L] = Tl
                    mult_to_point_K[new_tup_K] = Tk
                    for i in range(len(Tl)):
                        point_to_mult_L[Tl[i]] = new_tup_L
                        point_to_mult_K[Tk[i]] = new_tup_K
                    repeated = len(Tl)
                    if repeated not in repeated_mult_L:
                        repeated_mult_L[repeated] = [Tl]
                    else:
                        repeated_mult_L[repeated] += [Tl]
                    if repeated not in a_repeated_mult:
                        a_repeated_mult[repeated] = [Tl]
                    else:
                        a_repeated_mult[repeated] += [Tl]
                    # right now rechecking independence
                    source = []
                    corresponding = []
                    for r in sorted(repeated_mult_L.keys()):
                        for point_lst in repeated_mult_L[r]:
                            for point in point_lst:
                                if isLinearlyIndependent(source + [point]):
                                    source.append(point)
                                    mult = point_to_mult_L[point]
                                    # if another point with this multiplier and level pair is in S
                                    # then the multiplier level pair will be the last element of corresponding
                                    if len(corresponding) != 0:
                                        if corresponding[-1][0] == mult:
                                            corresponding[-1][1] += 1
                                        else:
                                            corresponding.append([mult, 1])
                                    else:
                                        corresponding.append([mult, 1])
                                if len(source) == n+2:
                                    more = False
                                    break
                            if len(source) == n+2:
                                break
                        if len(source) == n+2:
                            break
                if not more:
                    break
            if not more:
                break
        if found_no_more:
            raise ValueError('no more rational preimages')
        i_repeated_mult = deepcopy(a_repeated_mult)
        a_repeated_mult = {}
        found_no_more = True
        #print('i_repeated_mult:', i_repeated_mult)
    #print('source:', source)
    #print('mult_to_point_L:', mult_to_point_L)
    #print('mult_to_point_K:', mult_to_point_K)
    #print('point_to_mult_L:', point_to_mult_L)
    #print('point_to_mult_K:', point_to_mult_K)
    print('repeated_mult_L keys:', sorted(repeated_mult_L.keys()))
    possible_targets = []
    for tup in corresponding:
        possible_targets.append([mult_to_point_K[tup[0]], tup[1]])
    #print('possible_targets:', possible_targets)
    arrangements_iterators = []
    for lst in possible_targets:
        arrangements_iterators.append(Arrangements(range(len(lst[0])), lst[1]))
    print('arrangement_iterators:', arrangements_iterators)
    print('cardinalities:', [itr.cardinality() for itr in arrangements_iterators])
    count=0
    count2 = 0
    Conj = []
    for tup in product(*arrangements_iterators):
        #print('tup:', tup)
        current_target = []
        for i in range(len(tup)):
            for j in tup[i]:
                current_target.append(possible_targets[i][0][j])
        #print('current_target:', current_target)
        try:
            phi = f.domain().point_transformation_matrix(current_target, source)
            #print('phi:', phi)
            #print('phi inverse', phi.inverse())
            #print([phi*pnt for pnt in current_target])
            if self.conjugate(phi) == other:
                Conj.append(phi)
        except ValueError:
            count2 += 1
            pass
        count += 1
    print('number of targets tried:', count)
    print('number of not independent targets:', count2)
    return Conj

In [11]:
P.<x,y> = ProjectiveSpace(QQ, 1)
f = DynamicalSystem([x^2 + 3*y^2, 3*x*y+ y^2])
g = f.conjugate(matrix([[0, 1], [2, 2]]))
conj_set_preimage(f, g)

number of targets tried: 1


[
[  0 1/2]
[  1   1]
]

In [10]:
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([x^3, y^3, z^3])
conj_set_preimage(f, f)

number of targets tried: 360


[
[1 0 0]  [ 0  0  1]  [0 0 1]  [ 1  0  0]  [-1  0  0]  [ 0  0 -1]
[0 1 0]  [ 0 -1  0]  [0 1 0]  [ 0 -1  0]  [ 0  1  0]  [ 0 -1  0]
[0 0 1], [ 1  0  0], [1 0 0], [ 0  0  1], [ 0  0  1], [ 1  0  0],

[ 0  0 -1]  [-1  0  0]  [ 0 -1  0]  [ 0 -1  0]  [ 0  0 -1]  [ 0  0 -1]
[ 0  1  0]  [ 0 -1  0]  [-1  0  0]  [ 1  0  0]  [-1  0  0]  [ 1  0  0]
[ 1  0  0], [ 0  0  1], [ 0  0  1], [ 0  0  1], [ 0  1  0], [ 0  1  0],

[ 0  1  0]  [0 1 0]  [1 0 0]  [ 1  0  0]  [ 0 -1  0]  [ 0 -1  0]
[ 0  0 -1]  [0 0 1]  [0 0 1]  [ 0  0 -1]  [ 0  0 -1]  [ 0  0  1]
[ 1  0  0], [1 0 0], [0 1 0], [ 0  1  0], [ 1  0  0], [ 1  0  0],

[-1  0  0]  [-1  0  0]  [ 0  1  0]  [0 1 0]  [ 0  0  1]  [0 0 1]
[ 0  0  1]  [ 0  0 -1]  [-1  0  0]  [1 0 0]  [-1  0  0]  [1 0 0]
[ 0  1  0], [ 0  1  0], [ 0  0  1], [0 0 1], [ 0  1  0], [0 1 0]
]

In [4]:
len(l)

NameError: name 'l' is not defined

In [5]:
%%timeit
l = conj_set_preimage(f, f)

source: [(1 : 0 : 1), (-1 : 0 : 1), (0 : -1 : 1), (0 : 1 : 1)]
mult_to_point_L: {(x^2 - 3*x, 0): [(1 : 0 : 1), (-1 : 0 : 1), (0 : -1 : 1), (1 : 1 : 0), (-1 : 1 : 0), (0 : 1 : 1)], (x^2, 0): [(0 : 1 : 0), (1 : 0 : 0), (0 : 0 : 1)], (x^2 - 6*x + 9, 0): [(-1 : 1 : 1), (1 : -1 : 1), (-1 : -1 : 1), (1 : 1 : 1)]}
mult_to_point_K: {(x^2 - 3*x, 0): [(1 : 0 : 1), (-1 : 0 : 1), (0 : -1 : 1), (1 : 1 : 0), (-1 : 1 : 0), (0 : 1 : 1)], (x^2, 0): [(0 : 1 : 0), (1 : 0 : 0), (0 : 0 : 1)], (x^2 - 6*x + 9, 0): [(-1 : 1 : 1), (1 : -1 : 1), (-1 : -1 : 1), (1 : 1 : 1)]}
source: [(1 : 0 : 1), (-1 : 0 : 1), (0 : -1 : 1), (0 : 1 : 1)]
mult_to_point_L: {(x^2 - 3*x, 0): [(1 : 0 : 1), (-1 : 0 : 1), (0 : -1 : 1), (1 : 1 : 0), (-1 : 1 : 0), (0 : 1 : 1)], (x^2, 0): [(0 : 1 : 0), (1 : 0 : 0), (0 : 0 : 1)], (x^2 - 6*x + 9, 0): [(-1 : 1 : 1), (1 : -1 : 1), (-1 : -1 : 1), (1 : 1 : 1)]}
mult_to_point_K: {(x^2 - 3*x, 0): [(1 : 0 : 1), (-1 : 0 : 1), (0 : -1 : 1), (1 : 1 : 0), (-1 : 1 : 0), (0 : 1 : 1)], (x^2, 0): [(0 : 1 :

source: [(1 : 0 : 1), (-1 : 0 : 1), (0 : -1 : 1), (0 : 1 : 1)]
mult_to_point_L: {(x^2 - 3*x, 0): [(1 : 0 : 1), (-1 : 0 : 1), (0 : -1 : 1), (1 : 1 : 0), (-1 : 1 : 0), (0 : 1 : 1)], (x^2, 0): [(0 : 1 : 0), (1 : 0 : 0), (0 : 0 : 1)], (x^2 - 6*x + 9, 0): [(-1 : 1 : 1), (1 : -1 : 1), (-1 : -1 : 1), (1 : 1 : 1)]}
mult_to_point_K: {(x^2 - 3*x, 0): [(1 : 0 : 1), (-1 : 0 : 1), (0 : -1 : 1), (1 : 1 : 0), (-1 : 1 : 0), (0 : 1 : 1)], (x^2, 0): [(0 : 1 : 0), (1 : 0 : 0), (0 : 0 : 1)], (x^2 - 6*x + 9, 0): [(-1 : 1 : 1), (1 : -1 : 1), (-1 : -1 : 1), (1 : 1 : 1)]}
source: [(1 : 0 : 1), (-1 : 0 : 1), (0 : -1 : 1), (0 : 1 : 1)]
mult_to_point_L: {(x^2 - 3*x, 0): [(1 : 0 : 1), (-1 : 0 : 1), (0 : -1 : 1), (1 : 1 : 0), (-1 : 1 : 0), (0 : 1 : 1)], (x^2, 0): [(0 : 1 : 0), (1 : 0 : 0), (0 : 0 : 1)], (x^2 - 6*x + 9, 0): [(-1 : 1 : 1), (1 : -1 : 1), (-1 : -1 : 1), (1 : 1 : 1)]}
mult_to_point_K: {(x^2 - 3*x, 0): [(1 : 0 : 1), (-1 : 0 : 1), (0 : -1 : 1), (1 : 1 : 0), (-1 : 1 : 0), (0 : 1 : 1)], (x^2, 0): [(0 : 1 :

KeyboardInterrupt: 

In [13]:
%%time
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([x^3,x^2*y+y^3,z^3])
conj_set_preimage(f, f)

number of targets tried: 4
CPU times: user 44.6 ms, sys: 0 ns, total: 44.6 ms
Wall time: 43.1 ms


[
[1 0 0]  [ 1  0  0]  [-1  0  0]  [-1  0  0]
[0 1 0]  [ 0 -1  0]  [ 0  1  0]  [ 0 -1  0]
[0 0 1], [ 0  0  1], [ 0  0  1], [ 0  0  1]
]

In [12]:
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([x*y^3 - x*z^3, -x^3*y + y*z^3, x^3*z - y^3*z])
conj_set_preimage(f, f)

AttributeError: 'AlgebraicScheme_subscheme_projective_field_with_category' object has no attribute 'list'

In [14]:
f.rational_preimages(P(1,0,0))

[]

In [1]:
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([x*z+2*x^2, x^2+z*y + y^2,z^2+y^2])
print(f.is_morphism())
g = f.conjugate(matrix(3,3,[0,1,1,-1,0,1,1,1,1]))
conj_set_preimage(f, g)

True


NameError: name 'conj_set_preimage' is not defined

In [3]:
%%time
K.<v>=CyclotomicField(3)
P.<x,y,z> = ProjectiveSpace(K, 2)
f = DynamicalSystem([8*x^7 - 35*x^4*y^3 - 35*x^4*z^3 - 7*x*y^6 - 140*x*y^3*z^3 - 7*x*z^6, -7*x^6*y - 35*x^3*y^4 - 140*x^3*y*z^3 + 8*y^7 - 35*y^4*z^3 - 7*y*z^6,-7*x^6*z - 140*x^3*y^3*z - 35*x^3*z^4 - 7*y^6*z - 35*y^3*z^4 + 8*z^7])

CPU times: user 16.7 ms, sys: 3.84 ms, total: 20.5 ms
Wall time: 219 ms


In [4]:
auto2 = conj_set_preimage(f,f)

repeated_mult_L keys: [9, 12, 36]
arrangement_iterators: [Arrangements of the set [0, 1, 2, 3, 4, 5, 6, 7, 8] of length 4]
cardinalities: [3024]
number of targets tried: 3024
number of not independent targets: 1728


In [2]:
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([8*x^7 - 35*x^4*y^3 - 35*x^4*z^3 - 7*x*y^6 - 140*x*y^3*z^3 - 7*x*z^6, -7*x^6*y - 35*x^3*y^4 - 140*x^3*y*z^3 + 8*y^7 - 35*y^4*z^3 - 7*y*z^6,-7*x^6*z - 140*x^3*y^3*z - 35*x^3*z^4 - 7*y^6*z - 35*y^3*z^4 + 8*z^7])
conj_set_preimage(f,f)

repeated_mult_L keys: [3, 4, 6]
arrangement_iterators: [Arrangements of the set [0, 1, 2] of length 2, Arrangements of the set [0, 1, 2, 3] of length 2]
cardinalities: [6, 12]
number of targets tried: 72
number of not independent targets: 24


[
[1 0 0]  [0 0 1]  [0 1 0]  [0 0 1]  [0 1 0]  [1 0 0]
[0 1 0]  [0 1 0]  [1 0 0]  [1 0 0]  [0 0 1]  [0 0 1]
[0 0 1], [1 0 0], [0 0 1], [0 1 0], [1 0 0], [0 1 0]
]

In [6]:
P.<x,y,z,w,j,t,h> = ProjectiveSpace(QQ, 6)
f = DynamicalSystem([x^2, y^2, z^2, w^2, j^2, t^2, h^2])
auto = conj_set_preimage(f, f)

repeated_mult_L keys: [1, 7, 21, 35]
arrangement_iterators: [Arrangements of the set [0] of length 1, Arrangements of the set [0, 1, 2, 3, 4, 5, 6] of length 7]
cardinalities: [1, 5040]
number of targets tried: 5040
number of not independent targets: 0


In [5]:
P.<x,y,z,w,t> = ProjectiveSpace(QQ, 4)
f = DynamicalSystem([x^3, y^3, z^3, w^3, t^3])
auto = conj_set_preimage(f, f)

repeated_mult_L keys: [5, 16, 20, 40]
arrangement_iterators: [Arrangements of the set [0, 1, 2, 3, 4] of length 5, Arrangements of the set [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] of length 1]
cardinalities: [120, 16]
number of targets tried: 1920
number of not independent targets: 0


In [77]:
len(auto2)

216

In [7]:
Subsets?